# Load Dependencies

In [1]:
from iqoptionapi.stable_api import IQ_Option
import time, json
from datetime import datetime
from termcolor import colored

# Estabelecendo Conexão

In [2]:
API = IQ_Option('ilton.aws@gmail.com', '/ajV@xjzJ-3peBv')

API.connect()
API.change_balance('PRACTICE')  # PRACTICE / REAL

con_tests = 0
max_con_test = 10

while True:
    con_tests = con_tests + 1
    if API.check_connect() is False:
        print('Erro ao tentar se conectar!')
        API.connect()
    elif con_tests == max_con_test:
        print('(error)Número máximo de tentativas excedidas!')
        break
    else:
        print('Conexão estabelecida!')
        break

Conexão estabelecida!


# Requisitando opções em aberto

In [3]:
pairs = API.get_all_open_time()
payouts = API.get_all_profit()

# Filter Open Options

In [4]:
def filter_options(input_arr,target_arr=[]):
    for pair in input_arr:
        if input_arr[pair]['open'] is True:
            target_arr.append(pair)
    return target_arr

In [5]:
open_options = []
open_options = filter_options(pairs['binary'])
open_options = filter_options(pairs['crypto'], open_options)
open_options

['GBPUSD',
 'CADCHF',
 'EURNZD',
 'AUDCHF',
 'GBPNZD',
 'EURUSD',
 'USDCAD',
 'AUDJPY',
 'EURAUD',
 'GBPJPY',
 'EURGBP',
 'USDJPY',
 'GBPCHF',
 'GBPAUD',
 'AUDNZD',
 'GBPCAD',
 'USDNOK',
 'AUDUSD',
 'EURCAD',
 'EURJPY',
 'NZDUSD',
 'ETHUSD',
 'IOTUSD-L',
 'EOSUSD-L',
 'QTMUSD',
 'EOSUSD',
 'BCHUSD-L',
 'ATOMUSD-L',
 'BTCUSD',
 'TRXUSD',
 'ONTUSD-L',
 'XRPUSD-L',
 'XLMUSD-L',
 'ETHUSD-L',
 'ZECUSD',
 'LTCUSD',
 'BCHUSD',
 'NEOUSD-L',
 'BTCUSD-L',
 'OMGUSD',
 'TRXUSD-L',
 'ADAUSD-L',
 'DSHUSD',
 'LTCUSD-L',
 'XRPUSD',
 'BNBUSD-L',
 'ETCUSD']

# Definindo Configuração da estratégia de operação

In [6]:
options = ['call', 'put']
interval = 60 #in seconds (60 means 1minute)
min_occurrence = 4
min_payout = float(0.84)
spend_bid = int(1) #min bid working: 2, test 1
timeframe = 1
index = 0

# Get Candles

In [7]:
def get_direction(candle):
        if candle['close'] > candle['open']:
            return 'positive'
        elif candle['close'] == candle['open']:
            return 'neutro'
        else:
            return 'negative'

In [8]:
def history_direction_occur(candles):
    counter = 0
    direction = None
    for candle in reversed(candles):
        n_direction = get_direction(candle)
        
        if direction is None:
            direction = n_direction
            counter = counter + 1    
            next
        
        if direction == n_direction:
            counter = counter + 1
        else:
            break
    return counter, direction

In [9]:
candles_option = {}
for pair_option in open_options:
    candles = API.get_candles(ACTIVES=pair_option, interval=interval, count=min_occurrence*2, endtime=time.time())
    counter, direction = history_direction_occur(candles)
    
    bid = 'put' if direction == 'positive' else 'call'
    if counter < min_occurrence: bid = None

    if bid is not None: 
        bought, bid_msg = API.buy(spend_bid, pair_option, bid, timeframe)
    else:
        bought, bid_msg = None, None
        
    pair_option_str=colored('PAIR_OPTION:', color='cyan', on_color='on_grey')
    bid_str=colored('BID:', color='cyan', on_color='on_grey')
    counter_str=colored('COUNTER:', color='cyan', on_color='on_grey')
    bought_str=colored('BOUGHT:', color='cyan', on_color='on_grey')
    bid_msg_str=colored('BOUGHT:', color='cyan', on_color='on_grey')
    
    print(pair_option_str,pair_option,bid_str,bid,counter_str,counter,bought_str,bought,bid_msg_str,bid_msg)

PAIR_OPTION: GBPUSD BID: None COUNTER: 3 BOUGHT: None BOUGHT: None
PAIR_OPTION: CADCHF BID: None COUNTER: 2 BOUGHT: None BOUGHT: None
PAIR_OPTION: EURNZD BID: None COUNTER: 2 BOUGHT: None BOUGHT: None
PAIR_OPTION: AUDCHF BID: None COUNTER: 2 BOUGHT: None BOUGHT: None
PAIR_OPTION: GBPNZD BID: None COUNTER: 3 BOUGHT: None BOUGHT: None
PAIR_OPTION: EURUSD BID: None COUNTER: 2 BOUGHT: None BOUGHT: None
PAIR_OPTION: USDCAD BID: None COUNTER: 2 BOUGHT: None BOUGHT: None
PAIR_OPTION: AUDJPY BID: None COUNTER: 2 BOUGHT: None BOUGHT: None
PAIR_OPTION: EURAUD BID: None COUNTER: 2 BOUGHT: None BOUGHT: None
PAIR_OPTION: GBPJPY BID: None COUNTER: 3 BOUGHT: None BOUGHT: None
PAIR_OPTION: EURGBP BID: None COUNTER: 3 BOUGHT: None BOUGHT: None
PAIR_OPTION: USDJPY BID: put COUNTER: 4 BOUGHT: True BOUGHT: 7437317152
PAIR_OPTION: GBPCHF BID: call COUNTER: 5 BOUGHT: True BOUGHT: 7437317166
PAIR_OPTION: GBPAUD BID: None COUNTER: 3 BOUGHT: None BOUGHT: None
PAIR_OPTION: AUDNZD BID: None COUNTER: 2 BOUGHT: No

In [10]:
# for pair in pairs['binary']:
#     if pairs['binary'][pair]['open'] is True:# and payout[par] >= payout_min:  
#         bought, id = API.buy(spend_bid,pair,"put",timeframe)
#         if bought:
#             print("Ordem executada com sucesso!")
#         else:
#             print("Não foi possível executar a ordem!")

In [11]:
# candles_option = {}
# for pair_option in open_options:
#     candles = API.get_candles(ACTIVES=pair_option, interval=interval, count=min_occurrence*2, endtime=time.time())
#     counter, direction = history_direction_occur(candles)
    
#     bid = 'put' if direction == 'positive' else 'call'
#     if counter < min_occurrence: bid = None
#     bought, id = API.buy(spend_bid,pair,"put",timeframe)
#     if bid is not None: 
#         bought, bid_id = API.buy(spend_bid, pair, bid, timeframe)
#         print(bought, bid_id)